#### Memory

In [5]:
from langchain.agents import AgentState, create_agent
from langchain.agents.middleware import AgentMiddleware
from typing import Any

##### Step 1: Define the State

In [10]:
class CustomState(AgentState):
    user_preferences: dict | None = None

##### Step 2: Middleware that inserts style preference into system prompt

In [11]:
class StyleMiddleware(AgentMiddleware):
    state_schema = CustomState

    def before_model(self, state, runtime) -> dict[str, Any] | None:
        # handle either a dict or an AgentState-like object
        if isinstance(state, dict):
            prefs = state.get("user_preferences") or {}
        else:
            # AgentState or pydantic-like object: use getattr
            prefs = getattr(state, "user_preferences", {}) or {}

        style = prefs.get("style", "simple")

        # Return a dict that will be merged with the call input (inject system msg)
        return {
            "messages": [
                {"role": "system", "content": f"Respond in a {style} style."}
            ]
        }

##### Step 3: Use it

In [12]:
agent = create_agent(
    model="gpt-4o-mini",
    middleware=[StyleMiddleware()]
)

In [14]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "Explain transformers"}],
    "state": {"user_preferences": {"style": "technical"}}
})

In [15]:
response["messages"][-1].content

"Transformers are a type of model used in machine learning, especially in natural language processing. They help computers understand and generate human language. Here's a simple breakdown:\n\n1. **Attention Mechanism**: Transformers use a technique called “attention” to focus on different parts of the input data. This helps them understand the context better, as they can weigh the importance of each word in a sentence.\n\n2. **Encoder-Decoder Structure**: Transformers are usually made up of two parts:\n   - **Encoder**: This part takes the input (like a sentence) and processes it to create a summary or representation.\n   - **Decoder**: This part takes the summary from the encoder and generates output (like translating that sentence into another language).\n\n3. **Parallel Processing**: Unlike previous models that processed data step by step, transformers can process all the words in a sentence at the same time. This makes them faster and more efficient.\n\n4. **Applications**: Transf

#### 2nd Example 